# Projeto de PO-233 
Este projeto busca realizar análise do problema proposto pelo Data Science Challenge 2019 - ITA.

##### Alunos:
- Fernando Zanchitta
- Davi Xie
- Hugo Timóteo

##### Dados
Os dados podem ser adquiridos no site: https://www.kaggle.com/competitions/data-science-challenge-at-eef-2019/data



## Bibliotecas necessárias

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt


### Leitura dos conjuntos de dados:
Os dados são lidos a partir dos arquivos $.csv$ gerados pelo _SubSampling.py_.

In [5]:
f_train = pd.read_csv('../dataset/features-training-subsampled.csv')
f_test = pd.read_csv('../dataset/features-test-subsampled.csv')
t_train = pd.read_csv('../dataset/target-training-subsampled.csv')

In [6]:
f_train.columns

Index(['id', 'property_id', 'created_on', 'operation', 'property_type',
       'place_name', 'place_with_parent_names', 'country_name', 'state_name',
       'geonames_id', 'lat_lon', 'lat', 'lon', 'currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',
       'expenses', 'description', 'title', 'image_thumbnail', 'collected_on'],
      dtype='object')

# Limpeza e Organização

Para começar a análise exploratória dos dados deve ser feita a retirada de atributos e informações que não serão utilizadas pelo projeto, a fins de simplificação e coerência com o proposto pela disciplina.

Nesse sentido vamos:
- Retirar as colunas irrelevantes
- Retirar as colunas de id
- Retirar colunas redundantes

#### Colunas irrelevantes

In [9]:
for column in f_train.columns:
    print(column, ' : ', f_train[column].nunique())



id  :  171141
property_id  :  161548
created_on  :  1237
operation  :  1
property_type  :  4
place_name  :  4107
place_with_parent_names  :  5185
state_name  :  27
geonames_id  :  0
lat_lon  :  34675
lat  :  34199
lon  :  33454
currency  :  4
surface_total_in_m2  :  1395
surface_covered_in_m2  :  1161
floor  :  127
rooms  :  27
expenses  :  2432
description  :  150803
title  :  113639
image_thumbnail  :  155288
collected_on  :  38


Vamos verificar os valores de algumas colunas para investigar a retirada:

In [11]:
# select unique values of columns that contain less that 30 unique values:
filtered_columns = f_train.columns[f_train.nunique() < 30]

# Get unique values for filtered columns
unique_values = f_train[filtered_columns].apply(lambda x: x.unique()).to_dict()

# Print unique values for each column
for column, values in unique_values.items():
    print(column, ' : ', values)

operation  :  ['sell']
property_type  :  ['apartment' 'house' 'PH' 'store']
state_name  :  ['São Paulo' 'Rio de Janeiro' 'Rio Grande do Sul' 'Goiás' 'Paraná'
 'Paraíba' 'Santa Catarina' 'Mato Grosso do Sul' 'Minas Gerais' 'Ceará'
 'Distrito Federal' 'Mato Grosso' 'Bahia' 'Espírito Santo' 'Pernambuco'
 'Rio Grande do Norte' 'Alagoas' 'Amazonas' 'Piauí' 'Pará' 'Tocantins'
 'Sergipe' 'Maranhão' 'Rondônia' 'Outros países' 'Amapá' 'Acre']
geonames_id  :  [nan]
currency  :  ['BRL' 'MXN' 'USD' 'COP']
rooms  :  [ 1.  2. nan  4.  3.  5. 10.  8.  7. 17.  6. 11.  9. 13. 12. 20. 15. 18.
 16. 14. 25. 22. 19. 21. 27. 30. 40. 24.]


Podemos retirar os valores de _operation_ e _geonames_id_

In [13]:
f_train = f_train.drop(['operation','geonames_id'],axis=1)
f_test = f_test.drop(['operation','geonames_id'],axis=1)

#### Valores identificadores

Alguns atributos possuem valores únicos para cada evento, portanto não adicionam poder preditivo ao modelo. 

In [18]:
#check max count of each value in features:
# verify only non nan values
for column in f_train.columns:
    print(column, ' : ', f_train[column].value_counts().max())
    #print value that has max count
    print(f_train[column].value_counts().idxmax())

created_on  :  10928
2017-05-23
property_type  :  111567
apartment
place_name  :  8101
Praia Grande
place_with_parent_names  :  8089
|Brasil|São Paulo|Praia Grande|
state_name  :  120493
São Paulo
lat_lon  :  844
-30.0,-51.0
lat  :  844
-30.0
lon  :  902
-51.0
currency  :  171105
BRL
surface_total_in_m2  :  2368
0.0
surface_covered_in_m2  :  3060
70.0
floor  :  920
1.0
rooms  :  50895
1.0
expenses  :  1675
500.0
description  :  4592
Excelente Oportunidade!Entre em contato e agende já uma visita com um de nossos corretores. Os preços, valores e caraterísticas exibidos poderão sofrer mudanças sem aviso prévio, por este motivo todas as informações deverão ser confirmadas pelo departamento comercial. Para otimizar o seu atendimento tenha em mãos o número da referência do imóvel (código). Entre em contato preenchendo o formulário e informe sempre um telefone para receber todas as informações.
title  :  1016
Apartamentos Vender
image_thumbnail  :  62
https://thumbs4.properati.com/6/SjfE7uhxF

Podemos retirar os atributos: _id_ e _property_id_, pois se comportam como identificadores.

In [17]:
f_train = f_train.drop(['id','property_id'],axis=1)
f_test = f_test.drop(['id','property_id'],axis=1)

Por decisão de projeto, vamos retirar a coluna de _image_thumbnail_, uma vez que nossa proposta não é trabalhar com imagens.

In [19]:
f_train = f_train.drop(['image_thumbnail'],axis=1)
f_test = f_test.drop(['image_thumbnail'],axis=1)

In [20]:
f_train.columns

Index(['created_on', 'property_type', 'place_name', 'place_with_parent_names',
       'state_name', 'lat_lon', 'lat', 'lon', 'currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',
       'expenses', 'description', 'title', 'collected_on'],
      dtype='object')

#### Valores faltantes

# Análise Exploratória dos dados

Podemos deletar as colunas *operation* e *country_name* que representam dados redundantes.


# Engenharia de Atributos

# Escolha de Atributos

# Treinamento e Teste

# Avaliação do Modelo

# Considerações Finais